In [ ]:
hazard_type = {'Unsafe_Condition' :	0,
'Unsafe_Behavior':	1,
'NM':	2,
'FA':	3
}
hazard_kind ={ 'Absence(Improper) of safety device.':0,
'breaching procedures.':1,
'FA - Without PSIF':2,
'Failure to communicate':3,
'Falling objects':4,
'Fire hazard':5,
'Improper bonding':6,
'Improper Cylinders':7,
'Improper design.':8,
'Improper Ergonomics.':9,
'Improper Tools/Equip.':10,
'Improper work environment.':11,
'Improper/missing of barricading / guarding':12,
'Lack of commitment.':13,
'Lack of inspection.':14,
'Lack of maintenance.':15,
'Lack of resources.':16,
'Lighting':17,
'LOTOTO':18,
'NM - With PSIF':19,
'NM - Without PSIF':20,
'Not wearing PPE.':21,
'Oil Leakage':22,
'Poor housekeeping.':23,
'Unnecessary scaf.':24,
'Unsafe storage':25
}
Eelect_kind = {
    'light': 0 , 
    'motor' : 1 , 
    'transformer': 2 , 
    'generator' : 3 , 
    'electrical room': 4 , 
    'earthing' : 5 ,
    'Cable' :  6 ,
    'panel' :7 ,
    'elevator' :8 ,
    'sensor' :9, 
    'hvac' : 10}
hazard_level = {'low':0 , 'medium' :1,'high':2}


In [ ]:
# Labeling data
import pandas as pd
import io

# Load the uploaded CSV file
try:
    # Attempt to read with potential header issues handled
    # The snippet suggests no header, just lines of text.
    df = pd.read_csv('electrical_data.csv', header=None, names=['text'], encoding='utf-8', on_bad_lines='skip')
except Exception as e:
    # Fallback for encoding issues
    try:
        df = pd.read_csv('electrical_data.csv', header=None, names=['text'], encoding='utf-8-sig', on_bad_lines='skip')
    except:
        # Last resort fallback
        df = pd.read_csv('electrical_data.csv', header=None, names=['text'], encoding='latin1', on_bad_lines='skip')

# Define keywords for each label (English and Arabic) based on the user's list and common synonyms
keywords = {
    0: ['light', 'lamp', 'illumination', 'fixture', 'bulb', 'lux', 'كشاف', 'انارة', 'إنارة', 'إضاءة', 'اضاءة', 'لمبة', 'لمبات', 'نور'],
    1: ['motor', 'engine', 'موتور', 'ماتور', 'محرك'],
    2: ['transformer', 'محول', 'ترانس'],
    3: ['generator', 'مولد'],
    4: ['electrical room', 'electric room','substation', 'switchgear', 'غرفة كهرباء', 'غرفه كهرباء', 'حجرة كهرباء', 'محطة'],
    5: ['earthing', 'earth', 'ground', 'ارضي', 'أرضي', 'تأريض', 'ارس'],
    6: ['cable', 'wire', 'cord', 'wiring', 'كابل', 'كبل', 'سلك', 'وير', 'وصلة', 'وصله'],
    7: ['panel', 'board', 'box', 'breaker', 'لوحة', 'لوحه', 'علبة', 'علبه', 'قاطع', 'تابلوه', 'كبينه', 'كابينة', 'بواط' , 'سوكت', 'بلجة'],
    9: ['sensor', 'switch', 'detector', 'حساس', 'سينسور', 'مفتاح', 'limit', 'limt', 'valve', 'valv' , 'برشر' , 'ثرمو كابل']
}

def get_label(text):
    if not isinstance(text, str):
        return -1
    text_lower = text.lower()
    
    found_labels = []
    # Check all keywords
    for label, keys in keywords.items():
        for k in keys:
            if k in text_lower:
                found_labels.append(label)
                break 
    
    if not found_labels:
        return -1
    
    # Priority Strategy:
    # If multiple found, we need a tie-breaker. 
    # The user provided list is 0-9. 
    # Common overlaps: "Motor Cable" (1 vs 6), "Panel Switch" (7 vs 9), "Substation Room" (8 vs 4).
    # "Motor Cable" -> Hazard is usually the cable. Cable (6).
    # "Panel Switch" -> Hazard is switch. Sensor/Switch (9).
    # "Electrical Room Light" -> Hazard is Light. Lighting (0).
    
    # Simple strategy: prioritize component over location, and specific over general.
    # 0 (Lighting), 9 (Sensor), 6 (Cable), 7 (Panel), 1 (Motor), 2 (Transformer), 3 (Generator), 5 (Earthing), 4 (Room), 8 (Station).
    # Let's try to sort found_labels by this priority? 
    # Or just use the first found in the loop (which is 0-9).
    # Let's stick to 0-9 order for consistency unless obvious. 
    # Returning the first one found in the 0-9 iteration.
    return found_labels[0]

# Apply labeling
df['label'] = df['text'].apply(get_label)

# Save the labeled file
df.to_csv('labeled_kinOfHazard.csv', index=False)
print(f"Labeled {len(df)} rows. Distribution:\n{df['label'].value_counts()}")

In [ ]:
# Generating classifier model
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import json
import os

# --- 1. SETUP & DATA PREPARATION ---
file_path = r'C:\Users\walid\.vscode\titan_management\ai\full_maintenance_training_data.csv'
df = pd.read_csv(file_path)

df['text'] = df['text'].astype(str)
df = df.dropna(subset=['text', 'label'])

sentences = df['text'].values

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['label'].values)
num_classes = len(np.unique(labels))

# --- 2. TOKENIZATION ---
vocab_size = 3000
max_length = 30
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# --- 3. BUILD MODEL ---
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 32, input_length=max_length),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# --- 4. TRAIN ---
print(f"🚀 Training Model on {num_classes} categories...")
model.fit(padded, labels, epochs=20, validation_split=0.2, verbose=1)

# --- 5. EXPORT: KERAS .H5 (For TFJS) ---
print("\n💾 Saving Keras Model...")
model.save('kind_of_hazard_model.h5')

# --- 6. EXPORT: TENSORFLOW LITE (For Mobile) ---
print("\n📱 Converting to TensorFlow Lite...")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('kind_of_hazard_model.tflite', 'wb') as f:
    f.write(tflite_model)

# --- 7. EXPORT: METADATA (FIXED) ---
print("\n📝 Saving Metadata...")

# Save Tokenizer
vocab_path = r'C:\Users\walid\.vscode\titan_management\ai\vocab.json'
with open(vocab_path, 'w', encoding='utf-8') as f:
    json.dump(word_index, f)

# FIX: Convert numpy types to standard python types for JSON
# We convert the 'label' to string to be safe
mapping = {int(i): str(label) for i, label in enumerate(label_encoder.classes_)}

with open('label_mapping.json', 'w') as f:
    json.dump(mapping, f)

print("\n" + "="*40)
print("✅ COMPLETED SUCCESSFULLY!")
print("1. 'kind_of_hazard_model.tflite' -> Ready for Flutter")
print("2. 'vocab.json' & 'label_mapping.json' -> Ready for logic")
print("3. 'kind_of_hazard_model.h5' -> Ready for Web conversion")
print("="*40)

In [ ]:
# 2. Run the conversion
#!tensorflowjs_converter --input_format=keras maintenance_model.h5 web_model
!tensorflowjs_converter --input_format=keras "C:\Users\walid\.vscode\titan_management\ai\maintenance_model.h5" web_model
# 3. Zip the result so you can download it
!zip -r web_model.zip web_model

In [ ]:
# multi label 

import pandas as pd
import io

# Load the uploaded text file
# Using 'observations.txt'
try:
    with open('observations.txt', 'r', encoding='utf-8') as f:
        lines = f.readlines()
except Exception:
    # Fallback for encoding
    with open('observations.txt', 'r', encoding='latin1') as f:
        lines = f.readlines()

# Create DataFrame
df = pd.DataFrame(lines, columns=['text'])
df['text'] = df['text'].str.strip()
df = df[df['text'] != ''] # Remove empty lines

# Define keywords for each label (English and Arabic)
# Based on user request:
# 0: Lighting, 1: Motor, 2: Transformer, 3: Generator, 4: Elec Room, 
# 5: Earthing, 6: Cable, 7: Panel, 8: Substation, 9: Sensor
keywords = {
    0: ['light', 'lamp', 'illumination', 'fixture', 'bulb', 'lux', 'كشاف', 'انارة', 'إنارة', 'إضاءة', 'اضاءة', 'لمبة', 'لمبات', 'نور' ,'كشفات' , 'لمبه' ,'اضاءه','الاضاءه' ,'اناره'],
    1: ['motor', 'engine', 'موتور', 'ماتور', 'محرك' , 'روزتة' ,'كفر مروحه'],
    2: ['transformer', 'محول', 'ترانس'],
    3: ['generator', 'مولد'],
    4: ['electrical room', 'electric room', 'switchgear', 'غرفة كهرباء', 'غرفه كهرباء', 'حجرة كهرباء', 'substation','محطة'],
    5: ['earthing', 'earth', 'ground', 'ارضي', 'أرضي', 'تأريض', 'ارس'],
    6: ['cable', 'wire', 'cord', 'wiring', 'كابل', 'كبل', 'سلك', 'وير', 'وصلة', 'وصله'],
    7: ['panel', 'board', 'box', 'breaker', 'لوحة', 'لوحه', 'علبة', 'علبه', 'قاطع', 'تابلوه', 'كبينه', 'كابينة', 'بواط',  'VCS' , 'بانل'],
    8: ['ELevator', 'اسانسير', 'محطه'],
    9: ['sensor', 'switch', 'detector', 'حساس', 'سينسور', 'مفتاح', 'limit', 'limt', 'valve', 'روبسوتش', 'ثرمومتر' , 'ليفل' , 'برشر' , 'بالرادار' , 'الرادار'],
    10: ['HVAC' , 'مروحة' , 'تكييف' ],
}

def get_label(text):
    if not isinstance(text, str):
        return -1
    text_lower = text.lower()
    
    found_labels = []
    # Check all keywords
    for label, keys in keywords.items():
        for k in keys:
            if k in text_lower:
                found_labels.append(label)
                break 
    
    if not found_labels:
        return -1 # Unlabeled
    
    # Priority Strategy: Return the first one found in the 0-9 iteration order
    # This aligns with the user's provided list order.
    return found_labels[0]

# Apply labeling
df['label'] = df['text'].apply(get_label)

# Filter out unlabeled rows (optional, but requested "label list")
# Let's keep them but maybe sort so unlabeled are at the end? 
# No, let's just output the file.

# Save the labeled file
df.to_csv('labeled_observations.csv', index=False)
print(f"Labeled {len(df)} rows. Distribution:\n{df['label'].value_counts()}")

In [ ]:
# generate the multi output model
import pandas as pd
import numpy as np
import tensorflow as tf
import json
import os
from sklearn.utils import class_weight
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# --- 1. CONFIGURATION ---
VOCAB_SIZE = 10000
MAX_LEN = 120
EMBEDDING_DIM = 100

# --- 2. LOAD & CLEAN DATA ---
print("Loading data...")
df = pd.read_csv('observations_labeled_data.csv')

# 2.1 FIX COLUMN NAMES & TYPOS
if 'light' in df.columns: df.rename(columns={'light': 'electrical_kind'}, inplace=True)

df['text'] = df['text'].astype(str).str.strip()
if 'level' in df.columns:
    df['level'] = df['level'].astype(str).str.lower().str.strip()
if 'electrical_kind' in df.columns:
    df['electrical_kind'] = df['electrical_kind'].astype(str).str.strip()
    df.loc[df['electrical_kind'] == 'cable', 'electrical_kind'] = 'Cable'

# --- 3. MAPPINGS (REMOVED TYPE) ---
HAZARD_KIND_MAPPING = {
    'Absence(Improper) of safety device.': 0, 'breaching procedures.': 1, 'FA - Without PSIF': 2,
    'Failure to communicate': 3, 'Falling objects': 4, 'Fire hazard': 5, 'Improper bonding': 6,
    'Improper Cylinders': 7, 'Improper design.': 8, 'Improper Ergonomics.': 9, 'Improper Tools/Equip.': 10,
    'Improper work environment.': 11, 'Improper/missing of barricading / guarding': 12, 'Lack of commitment.': 13,
    'Lack of inspection.': 14, 'Lack of maintenance.': 15, 'Lack of resources.': 16, 'Lighting': 17,
    'LOTOTO': 18, 'NM - With PSIF': 19, 'NM - Without PSIF': 20, 'Not wearing PPE.': 21,
    'Oil Leakage': 22, 'Poor housekeeping.': 23, 'Unnecessary scaf.': 24, 'Unsafe storage': 25
}
ELEC_KIND_MAPPING = {
    'light': 0, 'motor': 1, 'transformer': 2, 'generator': 3, 'electrical room': 4,
    'earthing': 5, 'Cable': 6, 'panel': 7, 'elevator': 8, 'sensor': 9, 'hvac': 10
}
LEVEL_MAPPING = { 'low': 0, 'medium': 1, 'high': 2 }

# --- 4. FILL MISSING LABELS ---
def get_missing_labels(row):
    h = row['hazard_kind'] if row['hazard_kind'] in HAZARD_KIND_MAPPING else 'Lack of maintenance.'
    e = row['electrical_kind'] if row['electrical_kind'] in ELEC_KIND_MAPPING else 'panel'
    l = row['level'] if row['level'] in LEVEL_MAPPING else 'low'
    return pd.Series([h, e, l])

df[['hazard_kind', 'electrical_kind', 'level']] = df.apply(get_missing_labels, axis=1)

# --- 5. PREPARE INPUTS ---
def map_to_int(col, mapping):
    return df[col].map(mapping).fillna(0).astype(int).values

y_hazard = map_to_int('hazard_kind', HAZARD_KIND_MAPPING)
y_elec = map_to_int('electrical_kind', ELEC_KIND_MAPPING)
y_level = map_to_int('level', LEVEL_MAPPING)

tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token='<OOV>')
tokenizer.fit_on_texts(df['text'].astype(str))
X = pad_sequences(tokenizer.texts_to_sequences(df['text']), maxlen=MAX_LEN, padding='post', truncating='post')

# --- 6. CALCULATE SAMPLE WEIGHTS ---
def get_weights_dict(y_data):
    classes = np.unique(y_data)
    weights = class_weight.compute_class_weight('balanced', classes=classes, y=y_data)
    return dict(zip(classes, weights))

w_hazard = get_weights_dict(y_hazard)
w_elec = get_weights_dict(y_elec)
w_level = get_weights_dict(y_level)

def get_sample_weights(y_data, weight_dict):
    return np.array([weight_dict[y] for y in y_data])

sw_hazard = get_sample_weights(y_hazard, w_hazard)
sw_elec = get_sample_weights(y_elec, w_elec)
sw_level = get_sample_weights(y_level, w_level)

print("Sample weights calculated.")

# --- 7. MODEL DEFINITION (3 OUTPUTS) ---
input_text = layers.Input(shape=(MAX_LEN,), name='input_text')
x = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM)(input_text)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=False))(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.3)(x) 

# Outputs (Only 3 now)
out_hazard = layers.Dense(len(HAZARD_KIND_MAPPING), activation='softmax', name='out_hazard')(x)
out_elec = layers.Dense(len(ELEC_KIND_MAPPING), activation='softmax', name='out_elec')(x)
out_level = layers.Dense(len(LEVEL_MAPPING), activation='softmax', name='out_level')(x)

model = Model(inputs=input_text, outputs=[out_hazard, out_elec, out_level])

# --- 8. COMPILE & TRAIN ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
    loss='sparse_categorical_crossentropy', 
    metrics={
        'out_hazard': 'accuracy',
        'out_elec': 'accuracy',
        'out_level': 'accuracy'
    }
)

print("Training...")
model.fit(
    X, 
    [y_hazard, y_elec, y_level], 
    sample_weight=[sw_hazard, sw_elec, sw_level], 
    epochs=30,  # Recommended: 30-50 epochs
    batch_size=32,
    verbose=1
)

# --- 9. EXPORT ---
print("Converting to TFLite...")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

with open('smart_issue_model.tflite', 'wb') as f:
    f.write(tflite_model)
    
# Save Vocab & Labels
with open('vocab.json', 'w', encoding='utf-8') as f:
    json.dump(tokenizer.word_index, f)

def save_labels(mapping, filename):
    sorted_labels = [k for k, v in sorted(mapping.items(), key=lambda item: item[1])]
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(sorted_labels, f)

save_labels(HAZARD_KIND_MAPPING, 'labels_hazard.json')
save_labels(ELEC_KIND_MAPPING, 'labels_elec.json')
save_labels(LEVEL_MAPPING, 'labels_level.json')

print("Done! Files generated.")